# MarlimR3D Model using `emg3d`

### Note regarding runtime

The following environment variables were set before starting Jupyter:
```
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export OPENBLAS_NUM_THREADS=1
```
This ensures that our code runs only on one thread. CPU-time is therefore the same as walltime (or even a tiny fraction smaller).

In [1]:
import emg3d
import discretize
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt
%load_ext memory_profiler

In [2]:
%matplotlib notebook

## Load model

In [3]:
data = np.load('../marlim_comp.npz')  #  or 'marlim_orig.npz'
tres_h = data['res_h']
tres_v = data['res_v']

# We have to add an air layer
hz = np.r_[data['hz'], 20]

mesh_c = discretize.TensorMesh(
    [data['hx'], data['hy'], hz], x0=data['x0'])

# Add air resistivity
res_h_c = 1e8*np.ones(mesh_c.vnC)
res_v_c = 1e8*np.ones(mesh_c.vnC)
res_h_c[:, :, :-1] = tres_h
res_v_c[:, :, :-1] = tres_v
del data, tres_h, tres_v

mesh_c

TensorMesh: 90,172,895 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x    515    364,325.00    415,825.00    100.00    100.00    1.00
   y    563  7,490,049.00  7,546,349.00    100.00    100.00    1.00
   z    311     -6,200.00         20.00     20.00     20.00    1.00

In [4]:
# QC resistivities
mesh_c.plot_3d_slicer(np.log10(res_h_c), clim=[np.log10(0.32), np.log10(2000)])

# QC anisotropies
#mesh_c.plot_3d_slicer(np.sqrt(res_v_c/res_h_c))

<IPython.core.display.Javascript object>

## Load survey

In [5]:
ds = xr.load_dataset('../marlim_survey.nc', engine='h5netcdf')
# data = xr.load_dataset('../marlim_data.nc', engine='h5netcdf')

### Extract required info

In [6]:
# Use reciprocity: rec becomes src
src = [ds.rec_x, ds.rec_y, ds.rec_z,
       ds.rec_theta, ds.rec_dip]

# Use reciprocity: src becomes rec
rec_x = ds.data_il.src_x[::2]
rec_y_il = ds.data_il.src_y
rec_z_il = ds.data_il.src_z

# Ensure same coordinates
print(np.allclose(rec_x, ds.data_bs.src_x[::2]))

rec_y_bs = ds.data_bs.src_y
rec_z_bs = ds.data_bs.src_z

# Frequency
freqs = ds.freqs.values

True


## Computation mesh

In [7]:
# Get cell widths and origin in each direction
inp = {'freq': freqs.min(), 'max_domain': 50000}
minres = res_h_c.min()
maxres = res_v_c[res_v_c<1e7].max()

xx, x0 = emg3d.utils.get_hx_h0(
    res=[minres, maxres], fixed=src[0], domain=[src[0]-5000, src[0]+5000],
    min_width=mesh_c.hx[0], **inp)

yy, y0 = emg3d.utils.get_hx_h0(
    res=[minres, maxres], fixed=src[1], domain=[src[1]-2000, src[1]+1000],
    min_width=mesh_c.hy[0], **inp)

zz, z0 = emg3d.utils.get_hx_h0(
    res=[minres, 100, maxres], domain=[-2000, 0],
    min_width=mesh_c.hz[0], **inp)

# Create a TensorMesh instance.
mesh = discretize.TensorMesh([xx, yy, zz], x0=[x0, y0, z0])
mesh

   Skin depth (m/l-r)  [m] : 805 / 56616
   Survey domain       [m] : 385275 - 395275
   Calculation domain  [m] : 340275 - 440275
   Final extent        [m] : 339791 - 440759
   Min/max cell width  [m] : 100 / 100 / 13553
   Alpha survey/calc       : 1.000 / 1.420
   Number of cells (s/c/r) : 128 (100/28/0)

   Skin depth (m/l-r)  [m] : 805 / 56616
   Survey domain       [m] : 7515812 - 7518812
   Calculation domain  [m] : 7467812 - 7567812
   Final extent        [m] : 7464832 - 7569792
   Min/max cell width  [m] : 100 / 100 / 12749
   Alpha survey/calc       : 1.000 / 1.330
   Number of cells (s/c/r) : 64 (30/34/0)

   Skin depth (m/l/r)  [m] : 805 / 14235 / 56616
   Survey domain       [m] : -2000 - 0
   Calculation domain  [m] : -50000 - 50000
   Final extent        [m] : -54798 - 65515
   Min/max cell width  [m] : 20 / 20 / 12696
   Alpha survey/calc       : 1.000 / 1.240
   Number of cells (s/c/r) : 160 (101/58/1)



TensorMesh: 1,310,720 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x    128    339,790.90    440,759.10    100.00 13,553.04    1.42
   y     64  7,464,832.49  7,569,791.51    100.00 12,749.05    1.33
   z    160    -54,798.33     65,514.73     20.00 12,696.40    1.24

In [8]:
# Interpolate to new mesh
res_h = emg3d.utils.grid2grid(mesh_c, res_h_c, mesh, 'volume', log=True)
res_v = emg3d.utils.grid2grid(mesh_c, res_v_c, mesh, 'volume', log=True)

# Create model instance
model = emg3d.utils.Model(mesh, res_x=res_h, res_z=res_v)

# `emg3d` computation

In [ ]:
# Pre-allocate results-array
egd = np.zeros((2, rec_x.size, 6, 6), dtype=complex)
    
max_mem = 0
tot_time = 0
for ii, freq in enumerate(freqs):
    print(f" === Frequency {ii+1}/{freqs.size} :: {freq} Hz ===")
    # Source field
    sfield = emg3d.utils.get_source_field(mesh, src, freq, strength=0)
   
    mem = %memit -o efield, info = emg3d.solve(mesh, model, sfield, sslsolver=True, linerelaxation=True, verb=-1, return_info=True)
    tot_time += info['time']
    ram = mem.mem_usage[0] - mem.baseline
    if ram > max_mem:
        max_mem = ram    
    
    # Extract (interpolate) Ex-field at receiver locations from the emg3d result.
    for i, rec in enumerate(zip([rec_y_il, rec_y_bs], [rec_z_il, rec_z_bs])):
        egd[i, :, ii, 0] = emg3d.utils.get_receiver(mesh, efield.fx, (rec_x, rec[0], rec[1]))
        egd[i, :, ii, 1] = emg3d.utils.get_receiver(mesh, efield.fy, (rec_x, rec[0], rec[1]))
        egd[i, :, ii, 2] = emg3d.utils.get_receiver(mesh, efield.fz, (rec_x, rec[0], rec[1]))
        
time = f"{tot_time:.0f} s"
ram = f"{max_mem:.0f} MiB"
print(f"time: {time}; memory usage: {ram}")

 === Frequency 1/6 :: 0.125 Hz ===


In [ ]:
# Save the three lines
ds.data_il.data[::2, :, :] = egd[0, :, :, :].real  # Inline RE
ds.data_il.data[1::2, :, :] = egd[0, :, :, :].imag  # Inline IM

ds.data_bs.data[::2, :, :] = egd[1, :, :, :].real  # Broadside RE
ds.data_bs.data[1::2, :, :] = egd[1, :, :, :].imag  # Broadside IM

# Add info
ds.attrs['runtime'] = time
ds.attrs['n_procs'] = 1
ds.attrs['max_ram'] = ram
ds.attrs['n_cells'] = f"({mesh.nCx} x {mesh.nCy} x {mesh.nCz}) - {mesh.nC}"
ds.attrs['n_nodes'] = 'N/A'
ds.attrs['n_dof'] = 'N/A'
ds.attrs['extent'] = (f"x = {mesh.vectorNx[0]:.1f}-{mesh.vectorNx[-1]:.1f}; " # Total mesh extent
                      f"y = {mesh.vectorNy[0]:.1f}-{mesh.vectorNy[-1]:.1f}; "
                      f"z = {mesh.vectorNz[0]:.1f}-{mesh.vectorNz[-1]:.1f}")
ds.attrs['min_vol'] = f"{np.min(mesh.vol):.1f}"
ds.attrs['max_vol'] = f"{np.max(mesh.vol):.1f}"
ds.attrs['machine'] = "Laptop; i7-6600U CPU@2.6 GHz x4; 16 GB of memory, Ubuntu 18.04"
ds.attrs['version'] = f"emg3d v{emg3d.__version__}"
ds.attrs['date'] = datetime.today().isoformat()

# ADD A NOTE that this is preliminary
ds.attrs['NOTE'] = 'Preliminary results, not final'

# Save it under <{model}_{code}.nc>
ds.to_netcdf(f"../results/marlim_emg3d.nc", engine='h5netcdf')
ds

## Figure 4 from Correa and Menezes (2019), but noise-free data

In [ ]:
data = xr.load_dataset('../marlim_data.nc', engine='h5netcdf')

In [ ]:
# Line styles
ls = ['co', 'ys', 'm*', 'gd', 'rv', 'b^']

fig, axs = plt.subplots(3, 2, figsize=(9, 10), sharex=True, sharey=True)

# Loop over Inline/Broadside
for iii, datname in enumerate(['data_il', 'data_bs']):

    # Take absolute value
    tdat = np.abs(getattr(data, datname).data[::2, :, :] +
                  1j*getattr(data, datname).data[1::2, :, :])

    # Loop over components Ex, Ey, Ez
    for ii, comp in enumerate(data.components.values[:3]):

        plt.sca(axs[ii, iii])
        plt.title(f"{['Inline', 'Broadside'][iii]} :: {comp}")

        # Loop over frequencies
        for i, freq in enumerate(data.freqs.values):

            # Plot this component/frequency
            plt.plot(rec_x, tdat[:, i, ii], ls[i][0]+'-', label=f"{freq:4.3f}")
            plt.plot(rec_x, abs(egd[iii, :, i, ii]), ls[i][0]+'.')
            #plt.plot(rec_x, abs(tdat[:, i, ii]-abs(egd[iii, :, i, ii])), ls[i][0]+'-')

        plt.axhline(2e-15)
        plt.legend(title='f (Hz)', loc='lower center', ncol=3)
        plt.grid('on')
        plt.yscale('log')
        
        if ii == 2:
            plt.xlabel('Offset (m)')
        if iii == 0:
            plt.ylabel('Normalized E-field magnitude (V/Am$^2$)')
        else:
            axs[ii, iii].yaxis.set_ticks_position('right')
            axs[ii, iii].yaxis.set_label_position('right')

plt.tight_layout()
plt.show()

In [ ]:
emg3d.Report([discretize, xr])